In [1]:
import pygame
#import matlab.engine  #MATLAB
import scapy.all as scapy
import json
import binascii
import os
from sys import exit

pygame 2.5.2 (SDL 2.28.2, Python 3.11.6)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
#Delete all the files

if os.path.exists("dev2mob.pcap"):
    os.remove("dev2mob.pcap")

if os.path.exists("dev2mob.json"):
    os.remove("dev2mob.json")
    
if os.path.exists("mob2dev.json"):
    os.remove("mob2dev.json")

In [3]:
#Function to write from pcap file to json file (FROM THE DEVICE TO THE SMARTPHONE)
def pcap_to_json_dev_mob(pcap_file, output_json_file):
    packets = scapy.rdpcap(pcap_file)
    with open(output_json_file, 'a') as file:
        for packet in packets:
            payload = binascii.hexlify(packet[scapy.Raw].load).decode('utf-8')
            payload_hex = payload[4:8]

            payload_binary = bin(int(payload_hex, 16))[2:]

            payload_binary_full = (16-len(payload_binary)) * "0" + payload_binary


            idMobile_binary = payload_binary_full[:9] 
            battery_binary = payload_binary_full[9:16]

            idMobile_decimal = int(idMobile_binary, 2)
            battery_decimal = int(battery_binary, 2)


            packet_data = {
                "idMobile": idMobile_decimal,
                "battery": battery_decimal
            }

            json.dump(packet_data, file)

In [4]:
#Function to write from pcap file to json file (FROM THE SMARTPHONE TO THE DEVICE)
def pcap_to_json_mob_dev(pcap_file, output_json_file):
    
    packets = scapy.rdpcap(pcap_file)
    for packet in packets:
        payload = binascii.hexlify(packet[scapy.Raw].load).decode('utf-8')
    payload_hex = payload[4:8]

    payload_binary = bin(int(payload_hex, 16))[2:]

    payload_binary_full = (16-len(payload_binary)) * "0" + payload_binary


    idMobile_binary = payload_binary_full[:9] 
    command_binary = payload_binary_full[9:11]
    maxVel_binary = payload_binary_full[11:16]

    idMobile_decimal = int(idMobile_binary, 2)
    command_decimal = int(command_binary, 2)
    maxVel_decimal = int(maxVel_binary, 2)
    
    mobile_data = [idMobile_decimal, command_decimal, maxVel_decimal]

    packet_data = {
        "idMobile": idMobile_decimal,
        "command": command_decimal,
        "maxVel": maxVel_decimal
    }

    with open(output_json_file, 'w') as json_file:
        json.dump(packet_data, json_file, indent=4)
    
    return mobile_data
    

In [5]:
#Function to write json file LIVE (FROM THE DEVICE TO THE SMARTPHONE)
def create_json_dev_mob(idMobile, battery, output_json_file):
    with open(output_json_file, 'w') as file:
        packet_data = {
            "idMobile": idMobile,
            "battery": battery
        }
        json.dump(packet_data, file)

In [6]:
pygame.init()

#Data from the device
curr_speed = 0
id_mobile = 0
max_speed = 31
battery = 100
brake_order = False

#Engine for MATLAB
#eng = matlab.engine.start_matlab() #MATLAB
#pcapObj = eng.get_writer("dev2mob") #MATLAB

#Data of the Simulation
screen = pygame.display.set_mode((1200,400))
pygame.display.set_caption("Simulation")
clock = pygame.time.Clock()
test_font = pygame.font.Font(None,30)

#Our bicycle object
bicycle = pygame.Surface((50,25)).convert_alpha() #regular surface
bicycle.fill("Green") #Fill the surface with red color
bicycle_rect = bicycle.get_rect(midleft=(0,200))

#Our speed text indicator
speed_surf = test_font.render("Speed = "+str(curr_speed), False, "Green")
speed_rect = speed_surf.get_rect(center=(600,50))

#Our obstacle
obstacle = pygame.Surface((50,150)).convert_alpha()
obstacle.fill("Red")
obstacle_rect = obstacle.get_rect(midleft=(900,200))

#The background
background = pygame.Surface((1200,400))

#Timer for sending packets
packet_timer = pygame.USEREVENT + 1
pygame.time.set_timer(packet_timer,5000)

#Timer for decreasing the battery level
battery_timer = pygame.USEREVENT + 2
pygame.time.set_timer(battery_timer,5000)

#Timer for reading the pcap packet
read_timer = pygame.USEREVENT + 3
pygame.time.set_timer(read_timer,7000)

In [7]:
#Our sensors object. They are stacked right to each other to make them seem a triangle
sensors = []
sensors_rect = []

for i in range(40):
    sensor = pygame.Surface((5,2*i+1)).convert_alpha()
    sensor.fill("Yellow")
    sensors.append(sensor)
    if i == 0:
        sensor_rect = sensor.get_rect(midleft = bicycle_rect.midright)
        sensors_rect.append(sensor_rect)
    else:
        sensor_rect = sensor.get_rect(midleft = sensors_rect[i-1].midright)
        sensors_rect.append(sensor_rect)

In [8]:
#Main loop
while True:    
    #See if there has been any event
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            exit()
            
        if event.type == pygame.KEYDOWN and brake_order == False:
            if event.key == pygame.K_LEFT:
                #Decrease speed
                curr_speed = max(0, curr_speed - 1)
            if event.key == pygame.K_RIGHT:
                #Increase speed
                curr_speed = min(max_speed, curr_speed + 1)
            if event.key == pygame.K_ESCAPE:
                #End game
                pygame.quit()
                exit()
                
        if event.type == packet_timer:
            #Use matlab to send BLE packet
            #eng.create_frame(pcapObj,id_mobile,False,battery); #MATLAB
            print("PACKET")

            
        if event.type == battery_timer:
            battery = max(0, battery-2)
            create_json_dev_mob(id_mobile, battery, r"../MobileApp/packets.json")
            
            
        if event.type == read_timer:
            [id_read, command, max_read] = pcap_to_json_mob_dev("../BLE/mob2dev/mob2dev4.pcap","mob2dev.json")
            if id_mobile == id_read or id_mobile == 0:
                id_mobile = id_read
                max_speed = max_read
                if command == 3 and curr_speed > 0:
                    brake_order = True
                    
    #Apply the brakes if there is an order. Speed decreases by 1 in each frame
    if brake_order:
        curr_speed = max(0,curr_speed-1)
    
    if curr_speed == 0:
        brake_order = False

    #Draw the background on each frame
    screen.blit(background,(0,0))
    
    #Move the bicycle on each frame
    screen.blit(bicycle,bicycle_rect)
    bicycle_rect.left += curr_speed
    if bicycle_rect.left > 1200:
        bicycle_rect.left = 0
    
    #Display the current speed
    speed_surf = test_font.render("Speed = "+str(curr_speed), False, "Green")
    speed_rect = speed_surf.get_rect(center=(600,50))
    screen.blit(speed_surf,speed_rect)
    
    #Move the obstacle in the track
    screen.blit(obstacle,obstacle_rect)
    obstacle_rect.top += 1
    if obstacle_rect.top > 400:
        obstacle_rect.top = 0
    
    #Move the sensors 
    for i in range(len(sensors)):
        screen.blit(sensors[i],sensors_rect[i])
        if i == 0:
            sensors_rect[i].midleft = bicycle_rect.midright
        else:
            sensors_rect[i].midleft = sensors_rect[i-1].midright
    
    #Detect collision between sensors and the obstacle
    for i in range(len(sensors)):
        if sensors_rect[i].colliderect(obstacle_rect):
            curr_speed = min(curr_speed,i)
    
    #Update the display
    pygame.display.update()
    
    #Set a maximum framerate
    clock.tick(20)

PACKET
PACKET
PACKET
PACKET
PACKET
PACKET
PACKET
PACKET


SystemExit: 

/home/laura/.local/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3558: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
#pcap_to_json_dev_mob(r"dev2mob.pcap", "dev2mob.json")